In [9]:
# load data
import os
import pandas as pd
import re
from datetime import datetime
from dateutil.relativedelta import relativedelta
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import chart_studio
import chart_studio.plotly as py

from plotly.offline import plot
from plotly import graph_objects as go
from plotly import express as px

In [32]:
data_dir = os.getcwd() + "/data/"
files = os.listdir(data_dir)

txt_regex = re.compile('fred')

txt_file = list(filter(txt_regex.search, files))

save_path = os.getcwd() + "/plot/"
if os.path.isdir(save_path)  is not True:
    os.mkdir(save_path)

In [47]:
data_dir = os.getcwd() + "/data/"
files = os.listdir(data_dir)

txt_regex = re.compile('.pdf$')
png_file = list(filter(txt_regex.search, files))

In [49]:
data_dir

'/home/ec2-user/economy_stats/macro_index/src/data/'

In [6]:
txt = txt_file[0]
file = data_dir + txt
data = pd.read_csv(file, sep = "|")

# data type change
data.value = data.value.replace(".", None)
data = data.loc[data.value != ".",:].copy()
data.value = data.value.astype(float)

In [15]:
def draw_plotly_line(data):
    fig = px.line(data, x = 'date', y ='value')
    return fig

In [18]:
api_id = 'niceguy1575'
api_key = 'cAibgxtNqZuX1GOVb3km'

# draw plot
chart = draw_plotly_line(data)

# upload to char studio
chart_studio.tools.set_credentials_file(username = api_id, api_key = api_key)

chart_layer = txt
chart = py.plot(chart, filename = chart_layer, auto_open = False, fileopt = 'overwrite', sharing = 'public')


In [21]:
api_id = 'niceguy1575'
api_key = 'cAibgxtNqZuX1GOVb3km'

# text file
for txt in txt_file:
    file = data_dir + txt
    data = pd.read_csv(file, sep = "|")

    # data type change
    data.value = data.value.replace(".", None)
    data = data.loc[data.value != ".",:].copy()
    data.value = data.value.astype(float)
        
    # draw plot
    chart = draw_plotly_line(data)
    
    # upload to char studio
    chart_layer = txt
    chart_studio.tools.set_credentials_file(username = api_id, api_key = api_key)
    
    # 62, 65, 67, 69
    chart = py.plot(chart, filename = chart_layer, auto_open = False, fileopt = 'overwrite', sharing = 'public')

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['idx', 'value'] but received: date

In [13]:
# setup notion
import os
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import re
import requests
import time
from tzlocal import get_localzone


def requestURL(url, headers, requestType, data = None, retries=10):
    resp = None

    try:
        resp = requests.request(requestType, url, json = data, headers = headers)
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if 500 <= resp.status_code < 600 and retries > 0:
            print('Retries : {0}'.format(retries))
            return postUrl(url, param, retries - 1)
        else:
            print(resp.status_code)
            print(resp.reason)
            print(resp.request.headers)
    return resp


def BlockHeader(url, requestType, headers, headingType, text = None):
    
    heading = 'heading_1' if headingType == 1 else 'heading_2' if headingType == 2 else 'heading_3'
    
    data = {
    "children": [ {
          "type": heading,
          heading: {
            "rich_text": [{
              "type": "text",
              "text": {
                "content": text,
              }
            }],
            "color": "default"
          }
    } ] }
    
    return requestURL(url, headers, requestType, data)

def BlockParagraph(url, requestType, headers, text = None):
        
    data = {
    "children": [ {
          "type": "paragraph",
          "paragraph": {
            "rich_text": [{
              "type": "text",
              "text": {
                "content": text
              }
            }]
          } 
        } ] }
    
    return requestURL(url, headers, requestType, data)

def BlockEmbed(url, requestType, headers, embedURL):
        
    data = {
    "children": [ {
              "type": "embed",
                "embed": {
                    "url": embedURL
              }
    } ] }
    
    return requestURL(url, headers, requestType, data)

def BlockBookmark(url, requestType, headers, bookmarkURL):
        
    data = {
    "children": [ {
              "type": "bookmark",
                "bookmark": {
                    "url": bookmarkURL
              }
    } ] }
    
    return requestURL(url, headers, requestType, data)

In [38]:
# 0. 필요 데이터 setup
now = datetime.now()
now_f = now.strftime("%Y-%m-%d")

notion_secret_key = "secret_SzUg5gSUlHSzQthfKAZZk1icFugF3dFypnAc02DN826"
target_page_id = "07cae222-fb62-4bc5-b402-e96c1c86ed70"

ice_url = 'https://plotly.com/~niceguy1575/77/'
fed_funds_url = 'https://plotly.com/~niceguy1575/81/'
t10yt2 = 'https://plotly.com/~niceguy1575/79/'
inflate_url = 'https://plotly.com/~niceguy1575/75/'

# 1. 사전에 notion에서 사전에 페이지 획득 필요!
post_url = "https://api.notion.com/v1/pages"
headers = {
    "Accept": "application/json",
    "Notion-Version": "2022-02-22",
    "Content-Type": "application/json",
    "Authorization": "Bearer " + notion_secret_key
}
page_data = {
"parent": { "page_id": target_page_id },
"properties": {
    "title": {
        "title": [{ "type": "text", "text": { "content": now_f } }]
        }
    },
"children": [ {
    "object": "block",
    "type": "paragraph",
    "paragraph": {
        "rich_text": [{ "type": "text", "text": { "content": "중요한 미국 경제 지표를 한눈에 살펴볼 수 있는 레포트" } }]
        }
} ] }

# create page data
requestURL(post_url, headers, "POST", data = page_data)
time.sleep(5)

####################|################
# child block 생성
####################################
# 새로 만들어진 page id 확인
get_url = "https://api.notion.com/v1/blocks/"  + target_page_id +  "/children?page_size=100"

id_request = requestURL(get_url, headers, "GET")
parent_parse = pd.json_normalize(id_request.json())['results']
children_df = pd.json_normalize(parent_parse[0])
makingIDSeries = children_df[children_df.has_children == True].tail(n=1).id
makingID = makingIDSeries.values[0]

####################################
# notion api upload
####################################
childrenURL = 'https://api.notion.com/v1/blocks/' + makingID + '/children'

BlockHeader(childrenURL, "PATCH", headers, headingType = 2, text ='중요 거시경제 지표 확인')

# 1. 부도위험
BlockHeader(childrenURL, "PATCH", headers, headingType = 3, text ='부도위험 (ICE)')

# 1-2. ice graph
BlockEmbed(childrenURL, "PATCH", headers, embedURL = ice_url)

<Response [200]>

In [43]:
BlockBookmark(childrenURL, "PATCH", headers, 'https://www.federalreserve.gov/monetarypolicy/fomccalendars.html')

<Response [200]>

In [44]:
os.system("touch " + pdf_url)

0.setup_package.py
1.get_data_from_fred.py
2.draw_chart_studio.py
2.draw test.ipynb
3.upload_to_notion.py
data
plot
Untitled.ipynb


0

In [54]:
now = datetime.now() + timedelta(days=1)
now_f = now.strftime("%Y-%m-%d")

notion_secret_key = "secret_SzUg5gSUlHSzQthfKAZZk1icFugF3dFypnAc02DN826"
target_page_id = "07cae222-fb62-4bc5-b402-e96c1c86ed70"


data_dir = os.getcwd() + "/data/"
files = os.listdir(data_dir)

txt_regex = re.compile('.pdf$')
pdf_file = list(filter(txt_regex.search, files))

# url list
ice_url = 'https://plotly.com/~niceguy1575/77/'
fed_funds_url = 'https://plotly.com/~niceguy1575/81/'
t10yt2_url = 'https://plotly.com/~niceguy1575/79/'
inflate_url = 'https://plotly.com/~niceguy1575/75/'
fomc_url = 'https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm'
fng_url = 'https://money.cnn.com/data/fear-and-greed'
roro_url = 'https://www.mvis-indices.com/indices/customised/atac-risk-on-risk-off-domestic'
pdf_url = pdf_file


In [55]:
pdf_url

['12fwd_031122.pdf']

In [58]:
pdf_file[0]

'12fwd_031122.pdf'

In [57]:
https://www.factset.com/hubfs/Website/Resources%20Section/Research%20Desk/Earnings%20Insight/EarningsInsight_12fwd_031122.pdf

['.ipynb_checkpoints',
 'fred_T10YIE.txt',
 'fred_BAMLH0A0HYM2.txt',
 'fred_T10Y2Y.txt',
 'fred_FEDFUNDS.txt',
 'FG_image.png',
 '12fwd_031122.pdf',
 'p0-37.png',
 'business_cycle.png']

In [60]:
pd.to_csv()

AttributeError: module 'pandas' has no attribute 'to_csv'

In [64]:

pd.DataFrame(["https://www.factset.com/hubfs/Website/Resources%20Section/Research%20Desk/Earnings%20Insight/EarningsInsight_031122.pdf"]).to_csv('url_pdf.txt', sep = '|')

In [74]:
data_dir = os.getcwd() + "/data/"
files = os.listdir(data_dir)

txt_regex = re.compile('pdf.txt$')
pdf_file = list(filter(txt_regex.search, files))
x = pd.read_csv(data_dir + pdf_file[0], sep = "|")
x['0'][0]

In [78]:
x['0'][0]

'https://www.factset.com/hubfs/Website/Resources%20Section/Research%20Desk/Earnings%20Insight/EarningsInsight_031122.pdf'

In [70]:
data_dir

'/home/ec2-user/economy_stats/macro_index/src/data/'

In [2]:
class SendEmail:

    def __init__(self, senderEmailServer, senderEmail, senderPW):
        try: 
            # 587 -> outlook port number
            self.smtp = smtplib.SMTP(senderEmailServer, 587)
            self.smtp.ehlo() # say Hello
            self.smtp.starttls() # TLS 사용시 필요
            self.smtp.login(senderEmail, senderPW) 
        except Exception as e:
            print(e)
            self.smtp = smtplib.SMTP(senderEmailServer, 587)
            self.smtp.ehlo() # say Hello
            # self.smtp.starttls() # TLS 사용시 필요
            self.smtp.login(senderEmail, senderPW) 
            
    def MailSender(self, message, subject, senderEmail, targetEmail):
        
        # send with message
        #msg = MIMEText(message) 

        # send with HTML
        msg = MIMEMultipart('alternative')
        msg['Subject'] = subject
        msg['From'] = senderEmail
        msg['To'] = targetEmail
        
        # attach HTML
        msg.attach( MIMEText(message, 'html') )

        self.smtp.sendmail(senderEmail, targetEmail, msg.as_string()) 
        self.QuitSMTP()
        
    def QuitSMTP(self):
        self.smtp.quit()
        

In [7]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

page_link = 'https://niceguy1575.notion.site/07cae222fb624bc5b402e96c1c86ed70'
    
senderEmail = 'mr.niceguy1575@gmail.com'
senderPW = 'elaxttzkdxbqppnd'

SenderMailServer = 'smtp.gmail.com'
SenderEmail = 'mr.niceguy1575@gmail.com'
SenderPW = 'elaxttzkdxbqppnd'
SMail = SendEmail(SenderMailServer, SenderEmail, SenderPW) # 수신 메일 설정 

TargetEmail = 'niceguy1575@naver.com'
Subject = "📧 " + " 경제 리포트"

Message = \
"""<html>
        <head>
            <meta charset="utf-8">
        </head>
        <body>
            <h2> 당신을 위한 오늘의 경제 리포트 👀 </h2>
            <a href="https://niceguy1575.notion.site/07cae222fb624bc5b402e96c1c86ed70/"> Visit Your Report! </a>
        </body>
    </html>
"""

SMail.MailSender(Message, Subject, SenderEmail, TargetEmail)

In [16]:
save_path = os.getcwd() + "/data"
log_path = os.getcwd() + "/log"
today = datetime.today()
today_str = str(today.strftime("%Y-%m-%d"))

In [17]:
idstr = "07cae222-fb62-4bc5-b402-e96c1c86ed70".replace("-", "") 
base = 'https://niceguy1575.notion.site/'

In [18]:
'https://niceguy1575.notion.site/' + today_str + '-' + idstr

'https://niceguy1575.notion.site/2022-03-13-07cae222fb624bc5b402e96c1c86ed70'

In [ ]:
    2022-03-13-559d658e078342278dec79c84abc82cc
	# delete data folder
	print("delete files! " + now_f)
	os.system("rm " + data_dir + "*")

In [21]:
data_dir = '/home/ec2-user/economy_stats/run/data/'
files = os.listdir(data_dir)

txt_regex = re.compile('mail')
mail_file = list(filter(txt_regex.search, files))
report_link = pd.read_csv(data_dir + mail_file[0], sep = "|")
page_link = report_link['0'][0]

In [22]:
page_link

'https://niceguy1575.notion.site/2022-03-13-9cc24186fd7644e38adf459fc2d4637f'